## Import Required Libraries

In [5]:
import time
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

## Launch Browser & Load Website

In [6]:
browser = webdriver.Chrome()
browser.get("https://stockmages.netlify.app")

## Scroll the Page to Load Images

In [7]:
for i in tqdm(range(0, 5_000_000, 1000)):
    browser.execute_script("window.scrollTo(0," + str(i) + ")")
    time.sleep(0.1)

 87%|████████████████████████████████████████████████████████████████████▏         | 4369/5000 [13:02<01:52,  5.59it/s]


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=55250): Read timed out. (read timeout=120)

## Parse Page Source with BeautifulSoup

In [8]:
 soup = BeautifulSoup(browser.page_source, "html.parser")


In [10]:
data = []

for sp in tqdm(soup.find_all("div", class_="container")):
    img_link = sp.find("img").get("src")
    tags = sp.find("span", class_="tag-color").text[7:].strip()

    likes = int(
        sp.find("div", class_="likes-comments")
        .find_all("span")[0]
        .text.strip()[:-6]
    )

    comments = int(
        sp.find("div", class_="likes-comments")
        .find_all("span")[1]
        .text.strip()[:-9]
    )

    data.append([img_link, tags, likes, comments])


100%|████████████████████████████████████████████████████████████████████████████| 9104/9104 [00:01<00:00, 5593.53it/s]


## Create DataFrame & Save Metadata

In [13]:
df = pd.DataFrame(
    data,
    columns=["img_link", "tags", "likes", "comments"]
)

df.to_csv("images.csv", index=False)


## Image Download Function

In [14]:
def download(link, img_path):
    res = requests.get(link)
    with open(img_path, "wb") as fd:
        fd.write(res.content)


## Download All Images

In [16]:
paths = []
img_ids = []

ref = 100000
i = 1

for link in tqdm(df["img_link"]):
    img_id = "A" + str(i + ref)[1:]     # Creating Image ID
    i += 1

    img_path = "Imgs/" + img_id + ".jpg"
    download(link, img_path)

    paths.append(img_path)
    img_ids.append(img_id)


 26%|████████████████████                                                          | 2338/9104 [11:42<33:54,  3.33it/s]


KeyboardInterrupt: 

## Save Image Paths & IDs